# Motivation

In this notebook I'm looking at the difference between the modifiers in Tum

# Init

In [ ]:
%load_ext autoreload
import pandas as pd
import numpy as np
import os
import sys 
import importlib

import plotly.express as px
import plotly.io as pio
from plotly.offline import plot
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import multiprocess as mp

import igraph as ig

# Gsea libraries
import gseapy as gp
from gseapy.plot import dualplot

SCRIPT_DIR = os.path.dirname(os.path.abspath("pcgna_processing.py"))
sys.path.append(os.path.dirname(SCRIPT_DIR))
sys.path.append('/Users/vlad/Documents/Code/York/iNet_v2/src/')

from NetworkAnalysis.ExperimentSet import ExperimentSet
from NetworkAnalysis.NetworkComp import NetworkComp
from NetworkAnalysis import GraphHelper as gh
from NetworkAnalysis.utilities import sankey_consensus_plot as sky
from NetworkAnalysis.utilities.helpers import save_fig, survival_plot, survival_comp
import NetworkAnalysis.utilities.clustering as cs

pio.templates.default = "ggplot2"

%matplotlib inline

In [ ]:
data_base = "../../data/"
base_path = "../../results/"
exp_folder_tumour = "network_I/tum/"  # "/integration_v2.1/tum/" - path from iNET

# figures_path = base_path + exp_folder_tumour + "Figures/"
figures_path = "tum_modifiers/"

vu_output = pd.read_csv(f"{data_base}/metadata/VU_clustering_v3.tsv", sep="\t", index_col="Sample")

# prep mut
tcga_mutations_df = pd.read_csv(f"{data_base}/tumour/mutations_tcga.csv")
tcga_mutations_df = tcga_mutations_df[tcga_mutations_df["count"] != 0].set_index("gene")

all_tum_tpms = pd.read_csv(f"{data_base}/tumour/TPMs_selected_genes_v3_13k_gc42.tsv", sep="\t", index_col="genes")

all_tum_tpms_v4 = pd.read_csv(f"{data_base}/tumour/tum_TPMs_selected_genes_gc42_all_v4.tsv", sep="\t", index_col="genes")

# tf list
tf_path = f"{data_base}/metadata/TF_names_v_1.01.txt"
if os.path.exists(tf_path):
    tf_list = np.genfromtxt(fname=tf_path, delimiter="\t", skip_header=1, dtype="str")

## Load experiment sets

In [ ]:
%autoreload 2

tum = ExperimentSet("tum", base_path, exp_folder_tumour, tcga_mutations_df, sel_sets = ["4K",], rel_path="../")
# p0 = ExperimentSet("p0", base_path, exp_folder_p0, tcga_mutations_df, sel_sets = ["4K"], rel_path="../")

%autoreload 2
tum.export_to_gephi(save=False)
# p0.export_to_gephi(save=False)

## Computed ModCon and MEV scores

In [ ]:
def worker(arg):
    obj, methname = arg[:2]
    _ = getattr(obj, methname)()
    return obj

In [ ]:
pool = mp.Pool(mp.cpu_count())

results = pool.map(worker, ((exp, "get_ModCon") for exp in tum.exps.values()))
tum.exps = {exp.type: exp for exp in results}

In [ ]:
for exp in tum.get_exps():
    # exp.nodes_df["ModCon_Rank"] = 0
    for modCon, value in exp.modCons.items():
        dmy = value.sort_values(by=["ModCon_{}".format(exp.type)], ascending=False).reset_index(names="Id").iloc[:100]
        dmy["Rank"] = dmy.index + 1
        dmy.set_index("Id", inplace=True)
        exp.nodes_df.loc[exp.nodes_df["Modularity Class"] == modCon, "ModCon_Rank"] = dmy["Rank"]
        exp.nodes_df["ModCon_Rank"] = exp.nodes_df["ModCon_Rank"].fillna(0)

tum.export_to_gephi(save=False)
tum.generate_Mevs()

# Comms Mutation stats

In [ ]:
std_exp = tum.exps["standard_4K_10TF"]

mut_genes = std_exp.mut_df[std_exp.mut_df["count"] > 0].index
expressed_genes = std_exp.tpm_df.index.values
len(mut_genes) / len(expressed_genes)

In [ ]:
dmy = []
sel_exp = tum.exps["norm3_4K_10TF"]
sort_col = "ModCon_{}".format(sel_exp.type)
for comm, value in sel_exp.modCons.items():
    modConGenes = value.sort_values(by=sort_col, ascending=False).iloc[:100]
    modConMutated = sel_exp.mut_df[sel_exp.mut_df.index.isin(modConGenes)].shape[0]
    dmy.append((comm, modConMutated / len(mut_genes)))

# Leiden score comparison

Looking at the leiden score comparison between experiments

In [ ]:
leiden_scores = tum.comb_leiden_scores()

# the TF = 50 contain experiments with the different modifiers
leiden_scores = leiden_scores.loc[leiden_scores["TF"] != "50"]

# Specific to the tum dataset
leiden_scores.loc[leiden_scores["Modifier"] == "beta", "Modifier"] = "Penalty"

# Figure for multiple TFs and 3 Leiden scores
fig = px.scatter(leiden_scores, x="Modifier", color="TF", y="ModularityScore", size="ModuleNum", facet_col="Leiden Rank", facet_col_wrap=4)

# Fir for one TF and 10 Leiden scores
fig = gh.plot_leiden(exps=tum, tf="6")

# fig.update_layout(font=dict(size=16))
# save_fig(name="LeidenMetrics_{}".format(label), fig=fig, base_path=figures_path, width=1400, height=600)

# Network Metrics

Between standard, norm and reward compare the network metrics: degree, pageRank, closeness, betwenees and IVI.

The network configuration is: 4K and 10TF. and 4K and 3TF.

## 4K 6TF (report)

In [ ]:
if False:
    std_nt, rwrd_nt, pen_nt = tum.exps["standard_4K_6TF"], tum.exps["norm3_4K_6TF"], tum.exps["beta_4K_6TF"]
    metrics_df = gh.prep_net_metrics(std_nt, rwrd_nt, pen_nt)

    fig = gh.plot_net_metrics(metrics_df, label="6TF", log_y=True, filename="NetworkMetricsComp_{}".format("6TF"), figs_path=figures_path)
    fig.show()

## 4K 10 TF

In [ ]:
if False:
    std_nt, rwrd_nt, pen_nt = tum.exps["standard_4K_10TF"], tum.exps["norm3_4K_10TF"], tum.exps["beta_4K_10TF"]
    metrics_df = gh.prep_net_metrics(std_nt, rwrd_nt, pen_nt)
    fig = gh.plot_net_metrics(metrics_df, label="10TF", filename="NetworkMetricsComp_{}".format("10TF"), figs_path=figures_path)

## Mutation representation (report)

In [ ]:
t_all_stats_df = gh.stats_mut_burden(all_tum_tpms, tf_list, tcga_mutations_df, type="All")
t_4k_stats_df = gh.stats_mut_burden(tum.exps["standard_4K_3TF"].tpm_df, tf_list, tcga_mutations_df, type="4K")

all_df = pd.concat([t_all_stats_df, t_4k_stats_df], axis=0)

In [ ]:
fig = gh.plot_mut_rep(all_df, title="Tumour. 4K top varied vs All expressed.")
fig = fig.update_layout(
    title="",
    legend=dict(
        orientation="h",
        title="Genes included",
        yanchor="bottom",
        xanchor="center",
        y=0.9,
        x=0.5,
        bgcolor="rgba(0,0,0,0)",
        font=dict(size=22, color="#003366"),
    ),
    xaxis=dict(tickfont=dict(size=18), title="Mutation burden"),
    yaxis=dict(tickfont=dict(size=18)),
    font=dict(size=18),
)
fig.update_yaxes(matches=None)
fig.show()
save_fig(name="MutTF_representation_4K-all", fig=fig, base_path=figures_path, width=1400, height=700, margin=0.02)

In [ ]:
mut_tf = all_df[(all_df.index.str.contains("TF")) & (all_df["Type"] == "4K")]
mut_tf["Prct"] = round(mut_tf["Num"] / 265, 2)

sel_4k_df = all_df[(~all_df.index.str.contains("TF")) & (all_df["Type"] == "4K")]
sel_4k_df["Prct"] = round(sel_4k_df["Num"] / 4000, 2)

comb_df = pd.concat([mut_tf, sel_4k_df], axis=0).reset_index(names="Gene Type")
comb_df.loc[comb_df["Gene Type"] == "Mut", "Gene Type"] = "4K"
comb_df.loc[comb_df["Gene Type"] == "Mut_TF", "Gene Type"] = "TF"

fig = px.bar(
    comb_df,
    x="Burden",
    # y="Prct",
    y="Prct",
    color="Gene Type",
    barmode="group",
    text_auto=True,
    height=700,
)

fig = fig.update_layout(
    title="",
    legend=dict(
        orientation="h",
        title="Genes included",
        yanchor="bottom",
        xanchor="center",
        y=0.9,
        x=0.5,
        bgcolor="rgba(0,0,0,0)",
        font=dict(size=22, color="#003366"),
    ),
    xaxis=dict(tickfont=dict(size=18), title="Mutation burden"),
    yaxis=dict(tickfont=dict(size=18), title="Ratio"),
    font=dict(size=18),
)
fig.show()
save_fig(name="MutTF_representation_4K_TF_prct", fig=fig, base_path=figures_path, width=1400, height=500, margin=0.02)

# TF 6 (report)

## Comp

### Norm3 and standard

In [ ]:
%autoreload 2
show_figures = False

std_norm3_comp_tf10 = NetworkComp(tum, 4, "standard_4K_6TF", "norm3_4K_6TF")
std_norm3_comp_tf10.diff_in_com()

if 0:
    comp_dict = std_norm3_comp_tf10.comp_ge_comm()
    if False:
        # TODO move this into NetworkComp
        for key, df in comp_dict.items():
            fig = px.box(df, x="Comm", y="Median", color="Comm", title="Tum Median values in communities for {}".format(key), points="all")
            fig.show()

        display(std_norm3_comp_tf10.sankey_plot())
        display(std_norm3_comp_tf10.com_mut_distrib(include_source=False))
        display(std_norm3_comp_tf10.membership_change(include_source=False))

if show_figures:
    dmy_df, meta_norm3 = std_norm3_comp_tf10.comb_mut_stats(direction="Target")
    # display(std_norm3_comp_tf10.plot_mut_evo(dmy_df, direction="Target"))
    display(NetworkComp.plot_corr_matrix_coms(meta_norm3, height=700, title="Tum derived. Corr matrix for standard", hide_up=True))

    dmy_df, meta_std = std_norm3_comp_tf10.comb_mut_stats(direction="Source")
    # display(std_norm3_comp_tf10.plot_mut_evo(dmy_df, direction="Source"))
    display(NetworkComp.plot_corr_matrix_coms(meta_std, height=700, title="Tum derived. Corr matrix for reward", hide_up=True))

## Cluster methods

In [ ]:
tf = 6
comb_std, _, _ = gh.run_clusters(tum.exps["standard_4K_{}TF".format(tf)], label="std_tf{}".format(tf))
comb_norm3, _, _ = gh.run_clusters(tum.exps["norm3_4K_{}TF".format(tf)], label="norm3_tf{}".format(tf), show_figs=False)
comb_norm3.drop(columns=["PC_1", "PC_2"], inplace=True)
comb_beta, _, _ = gh.run_clusters(tum.exps["beta_4K_{}TF".format(tf)], label="beta_tf{}".format(tf))
comb_beta.drop(columns=["PC_1", "PC_2"], inplace=True)

sel_exp = tum.exps["standard_4K_{}TF".format(tf)]
comb_tf6 = pd.concat([comb_std, comb_norm3, comb_beta, vu_output], axis=1).dropna()

In [ ]:
num = 6
cluster_model = "RawKMeans"
reorder_cols = [
    "TCGA408_classifier",
    "{}_CS_{}_std_tf{}".format(cluster_model, num, tf),
    "{}_CS_{}_norm3_tf{}".format(cluster_model, num, tf),
    "{}_CS_{}_beta_tf{}".format(cluster_model, num, tf),
    "2019_consensus_classifier",
]
sky.main(df=comb_tf6, reorder_cols=reorder_cols, title="Best for {}. Comp between {} ".format(sel_exp.type, ", ".join(reorder_cols)))

## Modularity score and Sankey

In [ ]:
tf, no_K, no_genes, cs_model = 6, 6, "4K", "RawKMeans"

tum, sky_fig, cols = gh.prep_sankey_leiden(tum, vu_output=vu_output, tf=tf, no_K=no_K, no_genes=no_genes, chosen_cs_model=cs_model)
sky_fig.update_layout(
    title="",
    font=dict(size=18),
)
leiden_sky = gh.plot_sankey_leiden(tum, sky_fig=sky_fig, rename_cols=cols, label="TF-{}".format(tf), tf=str(tf))


leiden_sky = leiden_sky.update_layout(title="", xaxis=dict(tickfont=dict(size=18)), yaxis=dict(tickfont=dict(size=18)), font=dict(size=18), height=900)
leiden_sky = leiden_sky.update_annotations(font_size=22)
save_fig(name="LeidenMetrics_Sankey_{}".format("TF-6"), fig=leiden_sky, base_path=figures_path, width=1400, height=1000)

# TF 10

Experiments with lower numbers of edges per TF

## Std vs Norm3

In [ ]:
%autoreload 2
std_norm3_comp_tf = NetworkComp(tum, 4, "standard_4K_10TF", "norm3_4K_10TF")
std_norm3_comp_tf.diff_in_com()

show_figures = False

if show_figures:
    comp_dict = std_norm3_comp_tf.comp_ge_comm()
    for key, df in comp_dict.items():
        fig = px.box(df, x="Comm", y="Median", color="Comm", title="Tum Median values in communities for {}".format(key), points="all")
        # fig.show()

    # display(std_norm3_comp_tf.sankey_plot())
    display(std_norm3_comp_tf.com_mut_distrib(include_source=True, binarySource=True))
    # display(std_norm3_comp_tf.membership_change(include_source=False))


if show_figures:
    dmy_df, meta_norm3 = std_norm3_comp_tf.comb_mut_stats(direction="Target")
    # display(std_norm3_comp_tf10.plot_mut_evo(dmy_df, direction="Target"))
    display(NetworkComp.plot_corr_matrix_coms(meta_norm3, height=700, title="Tum derived. Corr matrix for standard", hide_up=True))

    dmy_df, meta_std = std_norm3_comp_tf.comb_mut_stats(direction="Source")
    # display(std_norm3_comp_tf10.plot_mut_evo(dmy_df, direction="Source"))
    display(NetworkComp.plot_corr_matrix_coms(meta_std, height=700, title="Tum derived. Corr matrix for reward", hide_up=True))

In [ ]:
selModCon = tum.exps["norm3_4K_10TF"].modCons
for com in [16, 20, 21, 23]:
    print("* Com = {}".format(com))
    print(selModCon[com].index.values)
    # print("* Com_{} = {}".format(com, ", ".join(selModCon[com].index.values)))

### Mutation evolution

In [ ]:
if 0:
    dmy_df = std_norm3_comp_tf.comb_mut_stats(direction="Target")
    display(std_norm3_comp_tf.plot_mut_evo(dmy_df, direction="Target"))

    dmy_df = std_norm3_comp_tf.comb_mut_stats(direction="Source")
    display(std_norm3_comp_tf.plot_mut_evo(dmy_df, direction="Source"))

## Std vs beta

In [ ]:
%autoreload 2
std_beta_comp_tf = NetworkComp(tum, 4, "standard_4K_10TF", "beta_4K_10TF")
std_beta_comp_tf.diff_in_com()

show_figures = False
if show_figures:
    comp_dict = std_beta_comp_tf.comp_ge_comm()
    for key, df in comp_dict.items():
        fig = px.box(df, x="Comm", y="Median", color="Comm", title="Tum Median values in communities for {}".format(key), points="all")
        # fig.show()


    # display(std_beta_comp_tf.sankey_plot())
    display(std_beta_comp_tf.com_mut_distrib(include_source=True,  binarySource=True))
    # display(std_beta_comp_tf.membership_change(include_source=False))


if show_figures:
    std_beta_comp_tf.comp_df = std_beta_comp_tf.comp_df.fillna(0)
    dmy_df, meta_std = std_beta_comp_tf.comb_mut_stats(direction="Target")
    # display(std_norm3_comp_tf10.plot_mut_evo(dmy_df, direction="Target"))
    display(NetworkComp.plot_corr_matrix_coms(meta_std, height=700, title="Tum derived. Corr matrix for standard", hide_up=True))

    dmy_df, meta_beta = std_beta_comp_tf.comb_mut_stats(direction="Source")
    # display(std_norm3_comp_tf10.plot_mut_evo(dmy_df, direction="Source"))
    display(NetworkComp.plot_corr_matrix_coms(meta_beta, height=700, title="Tum derived. Corr matrix for beta", hide_up=True))

## Morpheus
### Export

In [ ]:
%autoreload 2 
if 0:
    no_genes = 100
    for exp in tum.get_exps():
        if "beta2" in exp.name or "beta3" in exp.name:
            continue

        sort_col = "ModCon_{}".format(exp.type)

        exp.mevsMut, data = exp.get_mevs(exp.tpm_df, exp.modCons, sort_col=sort_col, num_genes=100)
        exp.export_morpheus_mevs(vu_output, exp.name + "_mut_100", tum=True)

### Import

In [ ]:
def import_morpheus(path, col_name="PGCNA_cut"):
    df = pd.read_csv(path, sep="\t", skiprows=2, index_col="id")
    df.index.names = ["Sample"]
    df = df.transpose().rename(columns={"dendrogram_cut": col_name})
    return df

In [ ]:
if 0:
    m_path = "{}/Stats/morpheus/10TF/cut_{}/".format(tum.path, 6)

    morpheus_std = import_morpheus(path="{}/{}_g{}.gct".format(m_path, "std", 100), col_name="tum_std")
    morpheus_norm3 = import_morpheus(path="{}/{}_g{}.gct".format(m_path, "norm3", 100), col_name="tum_norm3")
    morpheus_beta = import_morpheus(path="{}/{}_g{}.gct".format(m_path, "beta", 100), col_name="tum_beta")
    dmy_df = pd.concat([morpheus_std["tum_std"], morpheus_norm3["tum_norm3"], morpheus_beta["tum_beta"], vu_output], axis=1).dropna()
    # dmy_df.drop(index=["dendrogram_cut"], axis=1, inplace=True)
    reorder_cols = [
        "KMeans_labels_5",
        "TCGA408_classifier",
        "tum_std",
        "tum_norm3",
        "tum_beta",
    ]
    sky.main(df=dmy_df, reorder_cols=reorder_cols, title="Cut 7, Comparison between " + ", ".join(reorder_cols))

    sel_samples = dmy_df[dmy_df["TCGA"] == "Luminal_infiltrated"].copy(deep=True)
    sel_samples = dmy_df.copy(deep=True)
    sel_samples["mut_count"] = tcga_mutations_df.sum(axis=0)

    px.histogram(sel_samples, x="TCGA408_classifier", y="mut_count")

## Clustering methods

Find the optimal clustering

### Modifiers - best results

* Standard
  * Best results are given by clusters 4-6

* Norm3
  * When I run the experimnts with clustering methods: `["Birch", "RawKMeans", "GaussianMixture", "Ward", "SpectralClustering", "Avg"]` and comparted the metrics the Kmeans 4-6 exhibited the best results
* Beta

In [ ]:
tf = 10
comb_std, _, _ = gh.run_clusters(tum.exps["standard_4K_{}TF".format(tf)], label="std_tf{}".format(tf))
comb_norm3, _, _  = gh.run_clusters(tum.exps["norm3_4K_{}TF".format(tf)], label="norm3_tf{}".format(tf), show_figs=False)
comb_norm3.drop(columns=["PC_1", "PC_2"], inplace=True)
comb_beta, _, _ = gh.run_clusters(tum.exps["beta_4K_{}TF".format(tf)], label="beta_tf{}".format(tf))
comb_beta.drop(columns=["PC_1", "PC_2"], inplace=True)

sel_exp = tum.exps["standard_4K_{}TF".format(tf)]
comb_tf10 = pd.concat([comb_std, comb_norm3, comb_beta, vu_output], axis=1).dropna()

### Compare the best results between norm3 and standard

On K-means comparisons:
 * From what I can see that there is no major difference when CS is set between 3-4, but from >5 there are large differences.

Avg:
* Can't make the difference at Basal split

Spectral Clustering 
* It can only split the Basal after CS > 7
* Apart from that it has a similar behaviour with Kmeans


In [ ]:
num = 6
cluster_model = "RawKMeans"
reorder_cols = [
    "KMeans_labels_5",
    "{}_CS_{}_std_tf{}".format(cluster_model, num, tf),
    "{}_CS_{}_norm3_tf{}".format(cluster_model, num, tf),
    "{}_CS_{}_beta_tf{}".format(cluster_model, num, tf),
]
# sky.main(df=comb_tf10, reorder_cols=reorder_cols, title="Best for {}. Comp between {} ".format(sel_exp.type, ", ".join(reorder_cols)))

# variations = ["_".join(col.split("_")[:2]) for col in comb_std.columns]
# set(variations)

# TF 50

## Median values for each community

In [ ]:
%autoreload 2
std_norm3_comp_tf50 = NetworkComp(tum, 4, "standard_4K_50TF", "norm3_4K_50TF")
std_norm3_comp_tf50.diff_in_com()
display(std_norm3_comp_tf50.sankey_plot())

if 0:
    comp_dict = std_norm3_comp_tf50.comp_ge_comm()
    for key, df in comp_dict.items():
        fig = px.box(df, x="Comm", y="Median", color="Comm", title="Tum. Median values in communities for {}".format(key), points="all")
        fig.show()

    display(std_norm3_comp_tf50.sankey_plot())
    display(std_norm3_comp_tf50.com_mut_distrib(include_source=True))
    display(std_norm3_comp_tf50.membership_change(include_source=True))

## Mutation evolution


In [ ]:
if 0:
    dmy_df = std_norm3_comp_tf50.comb_mut_stats(direction="Target")
    display(std_norm3_comp_tf50.plot_mut_evo(dmy_df, direction="Target"))

    dmy_df = std_norm3_comp_tf50.comb_mut_stats(direction="Source")
    std_norm3_comp_tf50.plot_mut_evo(dmy_df, direction="Source")

## Clustering

In [ ]:
tf = 50
comb_std, _, _ = gh.run_clusters(tum.exps["standard_4K_{}TF".format(tf)], label="std_tf{}".format(tf))
comb_norm3, _, _ = gh.run_clusters(tum.exps["norm3_4K_{}TF".format(tf)], label="norm3_tf{}".format(tf))
comb_norm3.drop(columns=["PC_1", "PC_2"], inplace=True)
comb_beta, _, _ = gh.run_clusters(tum.exps["beta_4K_{}TF".format(tf)], label="beta_tf{}".format(tf))
comb_beta.drop(columns=["PC_1", "PC_2"], inplace=True)

sel_exp = tum.exps["standard_4K_{}TF".format(tf)]
comb_tf50 = pd.concat([comb_std, comb_norm3, comb_beta, vu_output], axis=1).dropna()

In [ ]:
num = 6
cluster_model = "RawKMeans"
reorder_cols = [
    "KMeans_labels_5",
    "{}_CS_{}_std_tf{}".format(cluster_model, num, tf),
    "{}_CS_{}_norm3_tf{}".format(cluster_model, num, tf),
    "{}_CS_{}_beta_tf{}".format(cluster_model, num, tf),
]
# sky.main(df=comb_tf50, reorder_cols=reorder_cols, title="Best for {}. Comp between {} ".format(sel_exp.type, ", ".join(reorder_cols)))

# TF 3

## Cluster methods

In [ ]:
tf = 3
comb_std, _, _   = gh.run_clusters(tum.exps["standard_4K_{}TF".format(tf)], label="std_tf{}".format(tf))
comb_norm3, _, _  = gh.run_clusters(tum.exps["norm3_4K_{}TF".format(tf)], label="norm3_tf{}".format(tf))
comb_norm3.drop(columns=["PC_1", "PC_2"], inplace=True)
comb_beta, _, _  = gh.run_clusters(tum.exps["beta_4K_{}TF".format(tf)], label="beta_tf{}".format(tf))
comb_beta.drop(columns=["PC_1", "PC_2"], inplace=True)

sel_exp = tum.exps["standard_4K_{}TF".format(tf)]
comb_tf3 = pd.concat([comb_std, comb_norm3, comb_beta, vu_output], axis=1).dropna()

In [ ]:
num = 3
cluster_model = "RawKMeans"
reorder_cols = [
    "KMeans_labels_5",
    "{}_CS_{}_std_tf{}".format(cluster_model, num, tf),
    "{}_CS_{}_norm3_tf{}".format(cluster_model, num, tf),
    "{}_CS_{}_beta_tf{}".format(cluster_model, num, tf),
]
# sky.main(df=comb_tf3, reorder_cols=reorder_cols, title="Best for {}. Comp between {} ".format(sel_exp.type, ", ".join(reorder_cols)))

# Clutering comp across TF experiments

## Norm3 (Reward)

In [ ]:
comb_all_tfs = pd.concat(
    [comb_tf3, comb_tf6.drop(columns=vu_output.columns), comb_tf10.drop(columns=vu_output.columns), comb_tf50.drop(columns=vu_output.columns)], axis=1
)

reorder_cols = [
    "KMeans_labels_6",
    # "{}_CS_{}_std_tf{}".format(cluster_model, num, 6),
    "{}_CS_{}_norm3_tf{}".format(cluster_model, num, 3),
    "{}_CS_{}_norm3_tf{}".format(cluster_model, num, 6),
    "{}_CS_{}_norm3_tf{}".format(cluster_model, num, 10),
    "RawKMeans_CS_6",
    # "KMeans_labels_5",
    # "{}_CS_{}_norm3_tf{}".format(cluster_model, num, 50),
]
sky.main(df=comb_all_tfs, reorder_cols=reorder_cols, title="Best for {}. Comp between {} ".format("Norm3", ", ".join(reorder_cols)))

## TF 10 norm3 and standard

In [ ]:
comb_all_tfs = pd.concat([comb_tf6.drop(columns=vu_output.columns), comb_tf10, comb_tf50.drop(columns=vu_output.columns)], axis=1)
num = 6
cluster_model = "RawKMeans"
reorder_cols = [
    "TCGA408_classifier",
    "KMeans_labels_6",
    "{}_CS_{}_norm3_tf{}".format(cluster_model, num, 10),
    "{}_CS_{}_std_tf{}".format(cluster_model, num, 10),
    # "RawKMeans_CS_6",
    # "KMeans_labels_5",
    # "{}_CS_{}_norm3_tf{}".format(cluster_model, num, 50),
]
# for col in reorder_cols:
#     comb_all_tfs[col] = comb_all_tfs[col].astype(str)
title = "Bio+Kmeans_5 - Network_Standard - Network_Reward - TCGA"
meta, fig = sky.main(df=comb_all_tfs, reorder_cols=reorder_cols, title=title, retMeta=True)
fig.update_layout(height=700)
fig.show()

In [ ]:
# fig.update_layout(font=dict(size=16))
# save_fig(name="Sankey_comp_NetworkVsStandard_num-{}_TF-{}".format(num, 10), fig=fig, base_path=figures_path, width=1400, height=900)

## Exploring Community enrichment for Basal 3 splits

In [ ]:
transform_cols = ["TCGA408_classifier", "consensus", "Lund2017.subtype"]
for col in transform_cols:
    vu_output[col] = vu_output[col].astype("category")
    vu_output[col + "_num"] = vu_output[col].cat.codes

In [ ]:
cluster_model = "RawKMeans"
num = 6
sel_exp = tum.exps["norm3_4K_10TF"]
sel_model = "{}_CS_{}_norm3_tf{}".format(cluster_model, num, 10)
prep_df = comb_tf10.loc[comb_tf10[sel_model].isin([2.0, 5.0, 4.0])]  # select the inf enriched clusters

sel_cols = ["KMeans_labels_6", sel_model]
prep_df = prep_df[sel_cols]
prep_df["KMeans_labels_6_num"] = pd.factorize(prep_df["KMeans_labels_6"])[0]
prep_df.drop(columns=["KMeans_labels_6"], inplace=True)

prep_df = pd.concat(
    [
        prep_df,
        sel_exp.mevsMut,
    ],
    axis=1,
).dropna()


mevs_name = "mevs_{}.tsv".format("Basal3_norm3")
prep_df.transpose().to_csv("{}stats/{}".format(sel_exp.exps_path, mevs_name), sep="\t", index=True)

In [ ]:
comb_norm3 = gh.run_clusters(tum.exps["norm3_4K_{}TF".format(tf)], label="norm3_tf{}".format(tf))

In [ ]:
dmy_df = comb_tf10.loc[comb_tf10[sel_model].isin([2.0, 5.0, 4.0])]  # select the inf enriched clusters
dmy_df = pd.concat([dmy_df[sel_model], sel_exp.mevsMut], axis=1).dropna()

dmy_df = dmy_df.loc[dmy_df[sel_model] == 2.0][dmy_df.columns[1:]]
top_coms = {}
num_top = 10
cmn_coms = set()
for idx, value in dmy_df.iterrows():
    cmn_comns = set(value.sort_values(ascending=False)[:num_top].index.values) & set(cmn_coms)

In [ ]:
# prep_df.loc[prep_df[sel_model]==4.0, sel_model] = 7
# prep_df.loc[prep_df[sel_model]==5.0, sel_model] = 4.0
# prep_df.loc[prep_df[sel_model]==7.0, sel_model] = 5

In [ ]:
# sky.main(prep_df, reorder_cols=["KMeans_labels_6", sel_model], title="Com")

### Estimate score

In [ ]:
cluster = "{}_CS_{}_norm3_tf{}".format(cluster_model, num, 10)
comb_all_tfs[cluster] = comb_all_tfs[cluster].astype(str)
# comb_all_tfs.loc[((comb_all_tfs["KMeans_labels_6"] == "High IFNG") & (comb_all_tfs[cluster] == "2.0")), cluster ]  = "inf"
fig1 = cs.plot_meta_scores(comb_all_tfs, y_axis="IFNG_score", classification=cluster, size="infiltration_score")

fig1.update_layout(
    legend=dict(
        orientation="v",
        title="Network_KMeans6",
        yanchor="middle",
        # y=0.74,
        xanchor="right",
        # x=0.13,
        bgcolor="rgba(0,0,0,0)",
    ),
    height=500,
)
# meta, fig = sky.main(df=comb_all_tfs, reorder_cols=reorder_cols, title=title, retMeta=True)
# fig.show()
# fig1.show()

In [ ]:
pathways = {
    "Hemidosome": ["COL17A1", "LAMB3", "DST", "ITGB4", "LAMA3", "KRT14", "LAMC2", "ITGA6", "KRT5", "PLEC"],
    "Com_16": [
        "DEDD",
        "ATF6",
        "USP21",
        "B4GALT3",
        "UHMK1",
        "UAP1",
        "PEX19",
        "DUSP12",
        "ALDH9A1",
        "PPOX",
        "KIFAP3",
        "MPZL1",
        "F11R",
        "EEF1AKNMT",
        "IGSF8",
        "TSTD1",
        "RP11-297K8.2",
        "TOMM40L",
        "KLHDC9",
        "IGSF9",
        "PIGK",
        "CREG1",
        "MPC2",
        "DUSP23",
        "TSEN15",
        "HSPA6",
    ],
    #     "Com_20": ["YES1" "USP14" "AFG3L2" "RAB12" "SEH1L" "CHMP1B" "PPP4R1" "SMCHD1" "MPPE1" "IMPA2" "CHMP1B-AS1" "TWSG1" "TTC39C" "NCK2" "CABYR"],
    #     "Com_21": ["P4HTM" "NPRL2" "ABHD14A" "NT5DC2" "PCBP4" "CYB561D2" "ABHD14B" "ACY1" "TREX1" "SMIM4" "CHCHD10" "SPATC1L"],
    #     "Com_23": ["ID3" "ID2" "ID4" "DUSP2" "SMAD6" "IGFBP2" "BMP2"],
    # }
}
# 'Metallothioneins Bind Metals_R': ["MT2A","MT1F","MT1G","MT1X","MT1E"],}

In [ ]:
tpm_df = tum.exps["standard_4K_10TF"].tpm_df

for key, val in pathways.items():
    dmy_df = pd.DataFrame(index=val)
    dmy_df["mut_count"] = tcga_mutations_df.loc[tcga_mutations_df.index.isin(val)]["count"]
    dmy_df["ge_median"] = tpm_df.loc[tpm_df.index.isin(val)].median(axis=1)
    fig = px.bar(dmy_df, x=dmy_df.index, y="ge_median", color="mut_count", title="Mut count for {}".format(key))
    fig.show()

# Comparing between experiments

Curious to see what's the difference between the 2 modifiers. If Beta is better at separating the communities.

Note: I couldn't see too much of a change when beta is applied. In fact, it feels that beta is very similar to the norm3 modifier

In [ ]:
# Prep data
std_t = tum.exps["standard_4K_10TF"]
norm3_t = tum.exps["norm3_4K_10TF"]
beta_t = tum.exps["beta_4K_10TF"]

dmy_std = std_t.nodes_df.rename(columns={"Modularity Class": "std_4K_10TF"})
dmy_norm3 = norm3_t.nodes_df.rename(columns={"Modularity Class": "norm3_4K_10TF"})
dmy_beta = beta_t.nodes_df.rename(columns={"Modularity Class": "beta_4K_10TF"})

# combine data
test = pd.concat([dmy_std["std_4K_10TF"], dmy_beta["beta_4K_10TF"], dmy_norm3["norm3_4K_10TF"]], axis=1).fillna(-1)  # dmy_h3["norm3_h_5k"]], axis=1)
reoder_cols = ["std_4K_10TF", "norm3_4K_10TF", "beta_4K_10TF"]  # "norm3_h_5k"]

# plot
meta = sky.main(
    test,
    reorder_cols=reoder_cols,
    title="Community comparison between " + ", ".join(reoder_cols),
)

In [ ]:
sel_mut = tcga_mutations_df[(tcga_mutations_df.index.isin(norm3_t.tpm_df.index)) & (tcga_mutations_df["count"] > 2)]
print("Genes mutated", sel_mut.shape[0] / norm3_t.tpm_df.shape[0])

In [ ]:
community_sizes = test["norm3_4K_10TF"].value_counts()
small_coms = community_sizes[community_sizes < 100].index.values
small_coms = test[test["norm3_4K_10TF"].isin(small_coms)]
# dmy_df = test.copy(deep=True)
# dmy_df.loc[~dmy_df["norm3_4K_10TF"].isin(small_coms), "norm3_4K_10TF"] = -1

meta = sky.main(
    small_coms,
    reorder_cols=reoder_cols,
    title="Just small communities comparison between " + ", ".join(reoder_cols),
)